In [116]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk
import tkinter.font as tkf
from tkinter import messagebox
from tkinter import filedialog
import threading
import time

In [117]:
raw_df = pd.read_csv('Human_GeCKOv2_Library_A_09Mar2015.csv')

In [118]:
raw_df['count'] = 0

In [119]:
raw_df.columns

Index(['gene_id', 'UID', 'seq', 'Unnamed: 3', 'count'], dtype='object')

In [120]:
raw_df

gene_id           UID  \
0                                       A1BG  HGLibA_00001   
1                                       A1BG  HGLibA_00002   
2                                       A1BG  HGLibA_00003   
3                                       A1CF  HGLibA_00004   
4                                       A1CF  HGLibA_00005   
5                                       A1CF  HGLibA_00006   
6                                        A2M  HGLibA_00007   
7                                        A2M  HGLibA_00008   
8                                        A2M  HGLibA_00009   
9                                      A2ML1  HGLibA_00010   
10                                     A2ML1  HGLibA_00011   
11                                     A2ML1  HGLibA_00012   
12                                   A3GALT2  HGLibA_00013   
13                                   A3GALT2  HGLibA_00014   
14                                   A3GALT2  HGLibA_00015   
15                                    A4GALT  HGLibA_00016   
16                                    A4GALT  HGLibA_00017   
17                                    A4GALT  HGLibA_00018   
18                                     A4GNT  HGLibA_00019   
19                                     A4GNT  HGLibA_00020   
20                                     A4GNT  HGLibA_00021   
21                                      AAAS  HGLibA_00022   
22                                      AAAS  HGLibA_00023   
23                                      AAAS  HGLibA_00024   
24                                      AACS  HGLibA_00025   
25                                      AACS  HGLibA_00026   
26                                      AACS  HGLibA_00027   
27                                     AADAC  HGLibA_00028   
28                                     AADAC  HGLibA_00029   
29                                     AADAC  HGLibA_00030   
...                                      ...           ...   
65353  NonTargetingControlGuideForHuman_0971  HGLibA_65354   
65354  NonTargetingControlGuideForHuman_0972  HGLibA_65355   
65355  NonTargetingControlGuideForHuman_0973  HGLibA_65356   
65356  NonTargetingControlGuideForHuman_0974  HGLibA_65357   
65357  NonTargetingControlGuideForHuman_0975  HGLibA_65358   
65358  NonTargetingControlGuideForHuman_0976  HGLibA_65359   
65359  NonTargetingControlGuideForHuman_0977  HGLibA_65360   
65360  NonTargetingControlGuideForHuman_0978  HGLibA_65361   
65361  NonTargetingControlGuideForHuman_0979  HGLibA_65362   
65362  NonTargetingControlGuideForHuman_0980  HGLibA_65363   
65363  NonTargetingControlGuideForHuman_0981  HGLibA_65364   
65364  NonTargetingControlGuideForHuman_0982  HGLibA_65365   
65365  NonTargetingControlGuideForHuman_0983  HGLibA_65366   
65366  NonTargetingControlGuideForHuman_0984  HGLibA_65367   
65367  NonTargetingControlGuideForHuman_0985  HGLibA_65368   
65368  NonTargetingControlGuideForHuman_0986  HGLibA_65369   
65369  NonTargetingControlGuideForHuman_0987  HGLibA_65370   
65370  NonTargetingControlGuideForHuman_0988  HGLibA_65371   
65371  NonTargetingControlGuideForHuman_0989  HGLibA_65372   
65372  NonTargetingControlGuideForHuman_0990  HGLibA_65373   
65373  NonTargetingControlGuideForHuman_0991  HGLibA_65374   
65374  NonTargetingControlGuideForHuman_0992  HGLibA_65375   
65375  NonTargetingControlGuideForHuman_0993  HGLibA_65376   
65376  NonTargetingControlGuideForHuman_0994  HGLibA_65377   
65377  NonTargetingControlGuideForHuman_0995  HGLibA_65378   
65378  NonTargetingControlGuideForHuman_0996  HGLibA_65379   
65379  NonTargetingControlGuideForHuman_0997  HGLibA_65380   
65380  NonTargetingControlGuideForHuman_0998  HGLibA_65381   
65381  NonTargetingControlGuideForHuman_0999  HGLibA_65382   
65382  NonTargetingControlGuideForHuman_1000  HGLibA_65383   

                        seq    Unnamed: 3  count  
0      GTCGCTGAGCTCCGATTCGA  HGLibA_00076      0  
1      ACCTGTAGTTGCCGGCGTGC           NaN      0  
2      CGTCAGCGTCACATTGGCCA           N

In [121]:
list(raw_df.ix[0])

['A1BG', 'HGLibA_00001', 'GTCGCTGAGCTCCGATTCGA', 'HGLibA_00076', 0]

In [122]:
headers = ['gene_id', 'UID', 'seq', 'Reserved', 'count']
header_widths = [80, 150, 350, 100, 80]

In [123]:
#!/usr/bin/env python

"""bm_preproc.py: Boyer-Moore preprocessing."""

__author__ = "Ben Langmead"

import unittest


def z_array(s):
    """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s """
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s)-1)

    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            z[1] += 1
        else:
            break

    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1

    for k in range(2, len(s)):
        assert z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    z[k] += 1
                else:
                    break
            r, l = k + z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            zkp = z[k - l]
            if nbeta > zkp:
                # Case 2a: zkp wins
                z[k] = zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                z[k] = r - k + 1
    return z


def n_array(s):
    """ Compile the N array (Gusfield theorem 2.2.2) from the Z array """
    return z_array(s[::-1])[::-1]


def big_l_prime_array(p, n):
    """ Compile L' array (Gusfield theorem 2.2.2) using p and N array.
        L'[i] = largest index j less than n such that N[j] = |P[i:]| """
    lp = [0] * len(p)
    for j in range(len(p)-1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1
    return lp


def big_l_array(p, lp):
    """ Compile L array (Gusfield theorem 2.2.2) using p and L' array.
        L[i] = largest index j less than n such that N[j] >= |P[i:]| """
    l = [0] * len(p)
    l[1] = lp[1]
    for i in range(2, len(p)):
        l[i] = max(l[i-1], lp[i])
    return l


def small_l_prime_array(n):
    """ Compile lp' array (Gusfield theorem 2.2.4) using N array. """
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i+1:  # prefix matching a suffix
            small_lp[len(n)-i-1] = i+1
    for i in range(len(n)-2, -1, -1):  # "smear" them out to the left
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i+1]
    return small_lp


def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p, lp), small_l_prime_array(n)


def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset i, and given L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i < length
    if i == length - 1:
        return 0
    i += 1  # i points to leftmost matching position of P
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]


def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_l_prime) - small_l_prime[1]


def dense_bad_char_tab(p, amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table.  Table is indexed by offset
        then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0, len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab


class BoyerMoore(object):
    """ Encapsulates pattern and associated Boyer-Moore preprocessing. """

    def __init__(self, p, alphabet='ACGT'):
        # Create map from alphabet characters to integers
        self.amap = {alphabet[i]: i for i in range(len(alphabet))}
        # Make bad character rule table
        self.bad_char = dense_bad_char_tab(p, self.amap)
        # Create good suffix rule table
        _, self.big_l, self.small_l_prime = good_suffix_table(p)

    def bad_character_rule(self, i, c):
        """ Return # skips given by bad character rule at offset i """
        assert c in self.amap
        assert i < len(self.bad_char)
        ci = self.amap[c]
        return i - (self.bad_char[i][ci]-1)

    def good_suffix_rule(self, i):
        """ Given a mismatch at offset i, return amount to shift
            as determined by (weak) good suffix rule. """
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1  # i points to leftmost matching position of P
        if self.big_l[i] > 0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]

    def match_skip(self):
        """ Return amount to shift in case where P matches T """
        return len(self.small_l_prime) - self.small_l_prime[1]


class TestBoyerMoorePreproc(unittest.TestCase):

    def test_z_1(self):
        s = 'abb'
        #    -00
        z = z_array(s)
        self.assertEqual([3, 0, 0], z)

    def test_z_2(self):
        s = 'abababab'
        #    00604020
        z = z_array(s)
        self.assertEqual([8, 0, 6, 0, 4, 0, 2, 0], z)

    def test_z_3(self):
        s = 'abababab'
        #    00604020
        z = z_array(s)
        self.assertEqual([8, 0, 6, 0, 4, 0, 2, 0], z)

    def test_n_1(self):
        s = 'abb'
        #    01-
        n = n_array(s)
        self.assertEqual([0, 1, 3], n)

    def test_n_2(self):
        s = 'abracadabra'
        #    1004010100-
        n = n_array(s)
        self.assertEqual([1, 0, 0, 4, 0, 1, 0, 1, 0, 0, 11], n)

    def test_n_3(self):
        s = 'abababab'
        #    0204060-
        n = n_array(s)
        self.assertEqual([0, 2, 0, 4, 0, 6, 0, 8], n)

    def test_big_l_prime_1(self):
        s = 'abb'
        #    001
        big_l_prime = big_l_prime_array(s, n_array(s))
        self.assertEqual([0, 0, 2], big_l_prime)

    def test_big_l_prime_2(self):
        s = 'abracadabra'
        #    01234567890
        # L' 00000003007
        # L  00000003337
        big_l_prime = big_l_prime_array(s, n_array(s))
        self.assertEqual([0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 8], big_l_prime)

    def test_small_l_prime_1(self):
        s = 'abracadabra'
        # N  1004010100-
        # l'           1
        # l'        4
        # l' 44444444111
        small_l_prime = small_l_prime_array(n_array(s))
        self.assertEqual([11, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1], small_l_prime)

    def test_good_suffix_match_mismatch_1(self):
        p = 'GGTAGGT'
        big_l_prime, big_l, small_l_prime = good_suffix_table(p)
        self.assertEqual([0, 0, 0, 0, 3, 0, 0], big_l_prime)
        self.assertEqual([0, 0, 0, 0, 3, 3, 3], big_l)
        self.assertEqual([7, 3, 3, 3, 3, 0, 0], small_l_prime)
        self.assertEqual(0, good_suffix_mismatch(6, big_l_prime, small_l_prime))
        self.assertEqual(0, good_suffix_mismatch(6, big_l, small_l_prime))
        #  t:      xT
        #  p: GGTAGGT
        # L': -000300
        #  L: -000333
        self.assertEqual(7, good_suffix_mismatch(5, big_l_prime, small_l_prime))
        self.assertEqual(4, good_suffix_mismatch(5, big_l, small_l_prime))
        #  t:     xGT
        #  p: GGTAGGT
        # L': -000300
        #  L: -000333
        self.assertEqual(7, good_suffix_mismatch(4, big_l_prime, small_l_prime))
        self.assertEqual(4, good_suffix_mismatch(4, big_l, small_l_prime))
        #  t:    xGGT
        #  p: GGTAGGT
        # L': -000300
        #  L: -000333
        self.assertEqual(4, good_suffix_mismatch(3, big_l_prime, small_l_prime))
        self.assertEqual(4, good_suffix_mismatch(3, big_l, small_l_prime))
        #  t:   xAGGT
        #  p: GGTAGGT
        # L': -000300
        #  L: -000333
        self.assertEqual(4, good_suffix_mismatch(2, big_l_prime, small_l_prime))
        self.assertEqual(4, good_suffix_mismatch(2, big_l, small_l_prime))
        #  t:  xTAGGT
        #  p: GGTAGGT
        # L': -000300
        #  L: -000333
        self.assertEqual(4, good_suffix_mismatch(1, big_l_prime, small_l_prime))
        self.assertEqual(4, good_suffix_mismatch(1, big_l, small_l_prime))
        #  t: xGTAGGT
        #  p: GGTAGGT
        # L': -000300
        #  L: -000333
        self.assertEqual(4, good_suffix_mismatch(0, big_l_prime, small_l_prime))
        self.assertEqual(4, good_suffix_mismatch(0, big_l, small_l_prime))

    def test_good_suffix_table_1(self):
        s = 'abb'
        #    001
        big_l_prime, big_l, small_l_prime = good_suffix_table(s)
        self.assertEqual([0, 0, 2], big_l_prime)
        self.assertEqual([0, 0, 2], big_l)
        self.assertEqual([3, 0, 0], small_l_prime)

    def test_good_suffix_table_2(self):
        s = 'abracadabra'
        #    01234567890
        # L' 00000003007
        # L  00000003337
        # l' -4444444111
        big_l_prime, big_l, small_l_prime = good_suffix_table(s)
        self.assertEqual([0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 8], big_l_prime)
        self.assertEqual([0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 8], big_l)
        self.assertEqual([11, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1], small_l_prime)

In [124]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

# Match

In [125]:
def countMatched():
    global indicator_match, sequences
    
    start_time = time.time()
    
    # Boyer-Moore Implementation///////////////////////
    num = len(sequences)
    indicator_match = 0
    gain = 1000000/num
    count = 0
    
    for sequence in sequences:    
        occurrences = boyer_moore(p, pbm, sequence)
        n = len(occurrences)
        count += n
        indicator_match += gain 
    # ///////////////////////////////////////////////
    
    end_time = time.time()
    delta_time = end_time - start_time
    
    text_count.delete('1.0', tk.END)
    text_count.insert('1.0', str(count))

    text_time.delete('1.0', tk.END)
    text_time.insert('1.0', str(delta_time))
    messagebox.showinfo("Matching Completed", "Matching count completed!")

In [126]:
def start_match_thread(event):
    global match_thread, indicator_match
    match_thread = threading.Thread(target=countMatched)
    match_thread.daemon = True
    
    progressbar_match['value'] = indicator_match
    match_thread.start()
    root.after(20, check_match_thread)

def check_match_thread():
    if match_thread.is_alive():
        progressbar_match['value'] = indicator_match
        root.after(20, check_match_thread)

# FASTQ File Load

In [127]:
def buttonBrowse():
    global filename
    try:
        filename = filedialog.askopenfilename(filetypes=(('FASTQ files', '*.fastq'), ('All files', '*.*')))
    except:
        filename = ''    

In [128]:
def readFastq():
    global sequences
    
    if filename == '':
        messagebox.showwarning("No File", "Sorry, no file loaded! Please choose FASTQ file first.")
    else:       

        f = open(filename)
        sequences = []

        try:
            while 1:
                name = f.readline().rstrip()
                sequence = f.readline().rstrip()
                f.readline()
                quality = f.readline().rstrip()

                if len(name) == 0:
                    break

                sequences.append(sequence)
            messagebox.showinfo("FASTQ File Loaded", "FASTQ file successfully loaded!")
        except:
            messagebox.showwarning("File Loading Failed", "Sorry, file loading failed! Please check the file format.")

In [129]:
def start_load_thread(event):
    global load_thread
    load_thread = threading.Thread(target=readFastq)
    load_thread.daemon = True
    
    progressbar_load.start(10)
    load_thread.start()
    root.after(20, check_load_thread)

def check_load_thread():
    if load_thread.is_alive():
        progressbar_load.start(10)
        root.after(20, check_load_thread)
    else:
        progressbar_load.stop()


# Table Events

In [130]:
def OnDoubleClick(event):
    item = table.selection()[0]
    value = table.item(item, 'values')
    name = value[0]
    
    

In [131]:
def sortby(tree, col, descending):
    """sort tree contents when a column header is clicked on"""
    # grab values to sort
    data = [(tree.set(child, col), child) for child in tree.get_children('')]
    # if the data to be sorted is numeric change to float
    #data =  change_numeric(data)
    # now sort the data in place
    data.sort(reverse=descending)
    for ix, item in enumerate(data):
        tree.move(item[1], '', ix)
    # switch the heading so it will sort in the opposite direction
    tree.heading(col, command=lambda col=col: sortby(tree, col, int(not descending)))

In [132]:
p = 'TCCTTTCCACAAGATATATA'

In [133]:
pbm = BoyerMoore(p)

# Main Flow

In [134]:
root = tk.Tk()

indicator_match = 0

root.geometry("{0}x{1}+0+0".format(root.winfo_screenwidth(), root.winfo_screenheight()))
#root.attributes('-fullscreen', True)
root.title('Sequence Matching Tool')


# Multicolumn Listbox/////////////////////////////////////////////////////////////////////////////
table = ttk.Treeview(height="20", columns=headers, selectmode="extended")
table.pack(padx=10, pady=20, ipadx=1200, ipady=190)

i = 1
for header in headers:
    table.heading('#'+str(i), text=header.title(), anchor=tk.W, command=lambda c=header: sortby(table, c, 0))
    table.column('#'+str(i), stretch=tk.NO, minwidth=0, width=tkf.Font().measure(header.title())+header_widths[i-1]) 
    i+=1    
table.column('#0', stretch=tk.NO, minwidth=0, width=0)

for a in range(len(raw_df)):
    table.insert("", "end", "", values=list(raw_df.ix[a]))

#table.insert('','end', values='')

table.bind("<Double-1>", OnDoubleClick)
#///////////////////////////////////////////////////////////////////////////////////////////

# Scrollbar////////////////////////////////////////////////////////////////////////////////////////
vsb = ttk.Scrollbar(table, orient="vertical",  command = table.yview)
hsb = ttk.Scrollbar(table, orient="horizontal", command = table.xview)
## Link scrollbars activation to top-level object
table.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
## Link scrollbar also to every columns
map(lambda col: col.configure(yscrollcommand=vsb.set,xscrollcommand=hsb.set), table)
vsb.pack(side = tk.RIGHT, fill = tk.Y)
hsb.pack(side = tk.BOTTOM, fill = tk.X)        

#//////////////////////////////////////////////////////////////////////////////////////////////



text_count=tk.Text(root, width=20,height=1, font=('tahoma', 9), bd=2)
text_count.place(x=60, y=750)
label_count=tk.Label(root, text='Count:', font=('tahoma', 9))
label_count.place(x=60,y=720)

text_time=tk.Text(root, width=20,height=1, font=('tahoma', 9), bd=2)
text_time.place(x=360, y=750)
label_time=tk.Label(root, text='Time:', font=('tahoma', 9))
label_time.place(x=360,y=720)

progressbar_match = ttk.Progressbar(root, length=500,  maximum=1000000, mode='determinate')
progressbar_match.place(x=520,y=600)

progressbar_load = ttk.Progressbar(root, length=300, mode='indeterminate')
progressbar_load.place(x=60,y=600)

button = ttk.Button(root, text="Match", width=20, command=lambda:start_match_thread(None))
button.place(x=520, y=650)

button_load = ttk.Button(root, text="Load", width=20, command=lambda:start_load_thread(None))
button_load.place(x=260, y=650)

button_load = ttk.Button(root, text="Browse...", width=20, command=buttonBrowse)
button_load.place(x=60, y=650)


root.bind('<Return>', start_match_thread)

root.mainloop()
